# Notebook for making calculating high risk area's

Make sure you have downloaded all the required datasets using the link provided in the GitHub repository. 

Place the files in the following folder structure (or update the paths in the notebook if different):

- `ffdi_dir`  → contains the monthly FFDI GeoTIFFs (`ffdi_YYYY_MM.tif`) | It should be present in Outputs folder
- `tif_dir`  → output folder for high risk rasters
- `study_area_path` → vector of your study area | Should be present in Data

Once the data is in place and paths are updated, you can run the notebook to calculate annual high-risk areas for the FFDI threshold and clip them to your study area.

**Runtime note:**  
Running this notebook on a geospatial computing platform took approximately **10 seconds** to complete for 11 years (2013–2023).

In [7]:
# import libraries
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.mask import mask as rio_mask
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import time

start_time = time.time()

In [12]:
# paths
ffdi_dir = "Outputs/ffdi"
os.makedirs(tif_dir, exist_ok=True)  # creates folder if it doesn't exist
tif_dir = "Outputs/highrisk_areas"
study_area_path = "Data/study_area.gpkg"

In [9]:
# settings
threshold = 12
start_year = 2013
end_year = 2023
calc_crs = "EPSG:3577"  # projected CRS for area calculation
out_crs = "EPSG:4326"   # output CRS for TIFs

# Load study area in calculation CRS
study_area = gpd.read_file(study_area_path).to_crs(calc_crs)

# Prepare dictionaries for results
annual_high_risk_area_whole = {}
annual_high_risk_area_study = {}

# Loop over years
for year in range(start_year, end_year + 1):
    monthly_files = sorted(glob.glob(os.path.join(ffdi_dir, f"ffdi_{year}_*.tif")))
    if not monthly_files:
        print(f"No rasters for {year}")
        continue

    yearly_mask_calc = None
    yearly_transform_calc = None

    # Combine monthly rasters into one high-risk mask
    for tif_path in monthly_files:
        with rasterio.open(tif_path) as src:
            transform, width, height = calculate_default_transform(
                src.crs, calc_crs, src.width, src.height, *src.bounds
            )
            data_reproj = np.empty((height, width), dtype=src.meta['dtype'])
            reproject(
                source=rasterio.band(src, 1),
                destination=data_reproj,
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=calc_crs,
                resampling=Resampling.nearest
            )

            mask_data = data_reproj >= threshold
            yearly_transform_calc = transform

            if yearly_mask_calc is None:
                yearly_mask_calc = mask_data
            else:
                yearly_mask_calc = np.logical_or(yearly_mask_calc, mask_data)

    # Calculate areas
    pixel_area_km2 = yearly_transform_calc.a * abs(yearly_transform_calc.e) / 1_000_000
    pixel_count_whole = np.sum(yearly_mask_calc)
    annual_high_risk_area_whole[year] = pixel_count_whole * pixel_area_km2

    # Study area mask for area calculation
    with rasterio.io.MemoryFile() as memfile:
        profile = {
            'driver': 'GTiff',
            'dtype': 'uint8',
            'count': 1,
            'height': yearly_mask_calc.shape[0],
            'width': yearly_mask_calc.shape[1],
            'crs': calc_crs,
            'transform': yearly_transform_calc
        }
        with memfile.open(**profile) as dataset:
            dataset.write(yearly_mask_calc.astype(rasterio.uint8), 1)
            clipped, _ = rio_mask(dataset, study_area.geometry, crop=True, all_touched=False)
            clipped_mask = clipped[0]

    pixel_count_study = np.sum(clipped_mask == 1)
    annual_high_risk_area_study[year] = pixel_count_study * pixel_area_km2

    # Reproject full raster to EPSG:4326
    dst_transform, dst_width, dst_height = calculate_default_transform(
        calc_crs, out_crs,
        yearly_mask_calc.shape[1], yearly_mask_calc.shape[0],
        left=0, bottom=0,
        right=yearly_mask_calc.shape[1] * yearly_transform_calc.a,
        top=yearly_mask_calc.shape[0] * abs(yearly_transform_calc.e)
    )
    dst_array = np.empty((dst_height, dst_width), dtype=rasterio.uint8)

    reproject(
        source=yearly_mask_calc.astype(rasterio.uint8),
        destination=dst_array,
        src_transform=yearly_transform_calc,
        src_crs=calc_crs,
        dst_transform=dst_transform,
        dst_crs=out_crs,
        resampling=Resampling.nearest
    )

    out_tif = os.path.join(tif_dir, f"ffdi_high_risk_{year}.tif")
    out_profile = {
        "driver": "GTiff",
        "dtype": "uint8",
        "count": 1,
        "width": dst_width,
        "height": dst_height,
        "crs": out_crs,
        "transform": dst_transform,
        "compress": "lzw"
    }
    with rasterio.open(out_tif, "w", **out_profile) as dst:
        dst.write(dst_array, 1)

    print(f"{year} — Whole: {annual_high_risk_area_whole[year]:,.2f} km², "
          f"Study: {annual_high_risk_area_study[year]:,.2f} km², saved to {out_tif}")

2013 — Whole: 1,714,523.75 km², Study: 2,927.06 km², saved to Outputs/highrisk_areas/ffdi_high_risk_2013.tif
2014 — Whole: 1,708,669.63 km², Study: 2,927.06 km², saved to Outputs/highrisk_areas/ffdi_high_risk_2014.tif
2015 — Whole: 1,653,787.31 km², Study: 2,927.06 km², saved to Outputs/highrisk_areas/ffdi_high_risk_2015.tif
2016 — Whole: 1,730,622.56 km², Study: 2,927.06 km², saved to Outputs/highrisk_areas/ffdi_high_risk_2016.tif
2017 — Whole: 1,795,749.59 km², Study: 2,927.06 km², saved to Outputs/highrisk_areas/ffdi_high_risk_2017.tif
2018 — Whole: 1,758,429.61 km², Study: 2,927.06 km², saved to Outputs/highrisk_areas/ffdi_high_risk_2018.tif
2019 — Whole: 1,841,118.97 km², Study: 7,317.64 km², saved to Outputs/highrisk_areas/ffdi_high_risk_2019.tif
2020 — Whole: 1,711,596.69 km², Study: 2,927.06 km², saved to Outputs/highrisk_areas/ffdi_high_risk_2020.tif
2021 — Whole: 1,710,133.16 km², Study: 2,927.06 km², saved to Outputs/highrisk_areas/ffdi_high_risk_2021.tif
2022 — Whole: 1,585

In [11]:
# --- Plot Whole Raster ---
plt.figure(figsize=(10,5))
plt.plot(
    list(annual_high_risk_area_whole.keys()),
    list(annual_high_risk_area_whole.values()),
    marker='o', color='red'
)
plt.xlabel("Year")
plt.ylabel("High-risk fire extent (km²)")
plt.title("Annual high-risk FFDI (≥12) – Whole raster")
plt.grid(True)
plt.tight_layout()
plt.savefig("high_risk_ffdi.png", dpi=300)
plt.close()

print("Successfully created plot: high_risk_ffdi.png")

# --- Plot Study Area ---
plt.figure(figsize=(10,5))
plt.plot(
    list(annual_high_risk_area_study.keys()),
    list(annual_high_risk_area_study.values()),
    marker='o', color='blue'
)
plt.xlabel("Year")
plt.ylabel("High-risk fire extent (km²)")
plt.title("Annual high-risk FFDI (≥12) – Study area")
plt.grid(True)
plt.tight_layout()
plt.savefig("high_risk_ffdi_study_area.png", dpi=300)
plt.close()

print("Successfully created plot: high_risk_ffdi_study_area.png")

Successfully created plot: high_risk_ffdi.png
Successfully created plot: high_risk_ffdi_study_area.png


In [13]:
end_time = time.time()
elapsed = end_time - start_time

print(f"✓ Script finished successfully in {elapsed:.2f} seconds")

✓ Script finished successfully in 162.31 seconds
